## Import Libraries

In [1]:
# %matplotlib ipympl
# %matplotlib inline
%matplotlib wx

In [2]:
import matplotlib.pyplot as plt
plt.ion()

In [3]:
from pydgilib_extra import *
from atprogram.atprogram import atprogram

In [4]:
from os import getcwd, path, pardir

## Compile and program project

In [5]:
project_path = [path.curdir, "BitCounting-S"]
project_path

['.', 'BitCounting-S']

In [6]:
atprogram(path.abspath(path.join(*project_path)), verbose=2)

make: Nothing to be done for 'all'.

[DEBUG] Starting execution of "chiperase"
[DEBUG] Starting process 'C:\Program Files (x86)\Atmel\Studio\7.0\atbackend\atbackend.exe'
[DEBUG] Connecting to TCP:127.0.0.1:21806
[INFO] Connected to edbg, fw version: 3.25
[INFO] Firmware check OK
[DEBUG] Command "chiperase" finished with return code 0
[DEBUG] Starting execution of "program"
[DEBUG] Memory segment base written at 0x00000000. Size = 0x00000904.
[DEBUG] Memory segment base written at 0x00000904. Size = 0x00000064.
[DEBUG] Memory segment base written at 0x0000fb00. Size = 0x00000020.
[DEBUG] Memory segment base written at 0x00804000. Size = 0x00000020.
[DEBUG] Command "program" finished with return code 0
[DEBUG] Exit successfully.
Firmware check OK
Chiperase completed successfully
Programming completed successfully.



0

## Data Logging

In [7]:
live_plot = True

Create a figure for the plot.

In [8]:
if live_plot:
    fig = plt.figure(figsize=(10, 6))
    fig.show()

Create the configuration dictionary for `DGILibExtra`.

In [9]:
config_dict = {
    "interfaces": [INTERFACE_POWER, INTERFACE_GPIO],
    "loggers": [LOGGER_OBJECT, LOGGER_CSV],
    "gpio_delay_time" : 0.0007,
    "file_name_base": "experiment_bit_counting"
}
config_dict_plot = {
    "loggers": [LOGGER_OBJECT, LOGGER_PLOT, LOGGER_CSV],
    "plot_pins": [True, True, True, True],
    "plot_pins_method": "line",
    "plot_xmax": 1,
    "window_title": "Experiment BitCounting",
}

Stop criteria to pass to the logger:

In [10]:
def stop_fn(logger_data):
    return all(logger_data.gpio.values[-1])

Stop criteria to pass to the parser:

In [11]:
def stop_function(pin_values):
    return all(pin_values)

In [12]:
repetitions = 10

Perform the measurement.

In [14]:
data = []
cd = config_dict.copy()
if live_plot:
    fig.clf()
    for ax in fig.get_axes():
        ax.cla()
    
    cd.update(config_dict_plot)
    cd["fig"] = fig
        
charges = []
times = []

with DGILibExtra(**cd) as dgilib:
    for _ in range(repetitions):
        dgilib.device_reset()
        dgilib.logger.log(2, stop_fn)
        print(dgilib.data)
        charge, time = power_and_time_per_pulse(dgilib.data, 2, stop_function=stop_function)
        charges += charge
        times += time
        if len(charge) > 1 or len(time) > 1:
            print(f"Parsing of measurement data failed, charge:{charge}, time:{time}")
            break
        dgilib.logger.plotobj.ax.set_title(f"Previous sample: charge: {charge[0]*1e3:.06} mC, time: {time[0]:.06} s")
        dgilib.empty_data()
    
    dgilib.logger.plotobj.ax.set_title(f"Average of {repetitions} samples: charge: {sum(charges)/len(charges)*1e3:.06} mC, time: {sum(times)/len(times):.06} s")

Interfaces:
	 256:   power,   samples:    3000
	  48:   gpio,    samples:       9

Interfaces:
	 256:   power,   samples:    3000
	  48:   gpio,    samples:       9

Interfaces:
	 256:   power,   samples:    3000
	  48:   gpio,    samples:       9

Interfaces:
	 256:   power,   samples:    3000
	  48:   gpio,    samples:       9

Interfaces:
	 256:   power,   samples:    3000
	  48:   gpio,    samples:       9

Interfaces:
	 256:   power,   samples:    3000
	  48:   gpio,    samples:      14

Interfaces:
	 256:   power,   samples:    3000
	  48:   gpio,    samples:       9

Interfaces:
	 256:   power,   samples:    3000
	  48:   gpio,    samples:       9

Interfaces:
	 256:   power,   samples:    3000
	  48:   gpio,    samples:      14

Interfaces:
	 256:   power,   samples:    3000
	  48:   gpio,    samples:       9



In [21]:
import json

config = {}
config["name"] = "Bit Counting 2048"
config["project_paths"] = [project_path]
config["config_dict"] = config_dict
config["config_dict_plot"] = config_dict_plot
config["analysis"] = {"pins":{2: ["Bit Counting"]}, 
                      "result_types": ["Charge", "Time"]}

with open("repeated_experiment.json", 'w') as config_file:  
    json.dump(config, config_file, indent=4)

In [14]:
import pandas as pd

In [15]:
df = pd.DataFrame({'times': times, 'charges':charges})

In [16]:
df

,times,charges
0,0.022706,0.000007
1,0.022639,0.000007
2,0.022640,0.000007
3,0.022705,0.000007
4,0.022639,0.000007
5,0.022639,0.000007
6,0.022639,0.000007
7,0.022637,0.000007
8,0.022703,0.000007
9,0.022703,0.000007


In [17]:
df.describe()

,times,charges
count,10.000000,1.000000e+01
mean,0.022665,6.619365e-06
std,0.000034,1.219877e-08
min,0.022637,6.592197e-06
25%,0.022639,6.614269e-06
50%,0.022640,6.620769e-06
75%,0.022703,6.627945e-06
max,0.022706,6.634523e-06


In [18]:
sum(charges)/repetitions

6.619364574372876e-06

In [19]:
print("%.0100f" % (sum(charges)/repetitions))

0.0000066193645743728758915470737622666064225995796732604503631591796875000000000000000000000000000000
